In [1]:
import numpy as np  

import sys
import subprocess
sys.path.append("/home/talax/xtof/local/Mod/lib64")
from mod import *
# from utils import smarts_to_gml_with_mapping
import random
from rdkit.Chem import AllChem,Draw
import numpy as np
from rdkit import Chem
# from rdchiral.main import rdchiralRun, rdchiralReaction, rdchiralReactants
import pandas as pd 
import torch
# from utils import smarts_to_gml

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::pair<double, double> already registered; second conversion method ignored.
/tmp/ipykernel_3605438/2816329013.py:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [42]:
from utils import collect_rules
rules_path = "/home/mescalin/yitao/Documents/Code/3HPspace"
rules = collect_rules(rules_path)
rule = rules[0]

for v in rule.vertices:
    # print(v.id)
    print(v.left.stringLabel)

    # print(v.right)
    # print(v.id)

for edge in rule.edges:
    print(edge.source.id)
    print(edge.target.id)
    print(edge.left.stringLabel)
    # print(edge.right.stringLabel)

C
C
O
C
O
O
H
H
NAD+
0
1
-
1
2
-
1
3
-
1
7
-
2
6
-
3
4
=
3
5
-
7
8


LogicError: Can not get string label on a null edge.

In [2]:
""" 

    Xe : indicate CoA group
    U : indicate NAD group
    Rn : indicate R group
"""

RULES  = [["[CH3:1][C:2]([H:8])([O:3][H:7])[C:4](=[O:5])[OH:6].[*+:9]>>[CH3:1][C:2](=[O:3])[C:4](=[O:5])[OH:6].[H+:7].[H:8][*:9]","(S)-lactate + NAD+ = pyruvate + NADH + H+","1.1.1.27"],
      ["[CH3:1][C:2]([H:6])=[O:3].[*+:5].[S:4]([H:8])[*:7]>>[CH3:1][C:2](=[O:3])[S:4][*:7].[H+:8].[*:5][H:6]","acetaldehyde + CoA + NAD+ = acetyl-CoA + NADH + H+","1.2.1.10"],
      ["[H:20][N:1]([H:21])[C:2]([CH2:3][C:4](=[O:5])[OH:6])([H:22])([C:7](=[O:8])[OH:9]).[O:10]=[C:11]([OH:12])[CH2:13][CH2:14][C:15](=[O:16])[C:17](=[O:18])[OH:19]>>[C:2]([CH2:3][C:4](=[O:5])[OH:6])([C:7](=[O:8])[OH:9])=[O:16].[H:20][N:1]([H:21])[C:15]([CH2:14][CH2:13][C:11](=[O:10])[OH:12])([H:22])[C:17](=[O:18])[OH:19]","L-Aspartate + 2-Oxoglutarate <=> Oxaloacetate + L-Glutamate","2.6.1.1"],
      ["[CH3:1](=[O:2])[S:3][*:5].[H:7][O:4][H:6]>>[CH3:1](=[O:2])[O:4][H:7].[H:6][S:3][*:5]","Acyl-CoA + H2O <=> CoA + Carboxylate","3.1.2.20"],
      ["[CH3:1][S+:2]([CH2:3][CH2:4][CH:5]([H:11])[NH2:6])[*:7].[O:8]=[C:9]=[O:10]>>[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C:9](=[O:10])[O:8][H:11])[*:7]","S-adenosyl-L-methionine = (5-deoxy-5-adenosyl)(3-aminopropyl)methylsulfonium salt + CO2","4.1.1.50"],
      ["[O:1]=[C:2]([OH:3])[CH:4]([H:11])[C@H:5]([O:6][H:10])[C:7](=[O:8])[OH:9]>>[O:1]=[C:2]([OH:3])/[CH:4]=[CH:5]/[C:7](=[O:8])[OH:9].[H:10][O:6][H:11]","fumarate + H2O = L-malate","4.2.1.2"],
      ["[H:8][CH2:1][CH:2]([O:3][H:9])[C:4](=[O:5])[S:6][*:7]>>[CH2:1]=[CH:2][C:4](=[O:5])[S:6][*:7].[H:8][O:3][H:9]","Lactoyl-CoA <=> Propenoyl-CoA + H2O","4.2.1.54"],
      ["[H:13][NH2:1][C@@H:2]([CH:3]([H:10])[C:4](=[O:5])[OH:6])[C:7](=[O:8])[OH:9]>>[CH:2](=[CH:3]/[C:4](=[O:5])[OH:6])\\[C:7](=[O:8])[OH:9].[NH2:1][H:10]","L-Aspartate <=> Fumarate + Ammonia","4.3.1.1"],
      ["[NH2:1][CH2:2][CH2:3][CH2:4][CH:5]([H:11])[C@H:6]([NH2:7])[C:8](=[O:9])[OH:10]>>[NH2:1][CH2:2][CH2:3][CH2:4][C@H:5]([NH2:7])[CH:6]([H:11])[C:8](=[O:9])[OH:10]","L-lysine = (3S)-3,6-diaminohexanoate","5.4.3.2"],
      ["[CH3:1][C:2](=[O:3])[OH:4].[Xe:18][S:5][H:19].[CH3:6][O:7][P:8](=[O:9])([OH:10])[O:11][P:12](=[O:13])([OH:14])[O:15][P:16](=[O:17])([OH:20])[OH:21]>>[CH3:1][C:2](=[O:3])[S:5][Xe:18].[OH:4][P:8]([O:7][CH3:6])(=[O:9])[OH:10].[O:11]([H:19])[P:12](=[O:13])([OH:14])[O:15][P:16](=[O:17])([OH:20])[OH:21]","ATP + Acetate + CoA <=> AMP + Diphosphate + Acetyl-CoA","6.2.1.1"],
      ["[H:23][CH2:1][C:2](=[O:3])[S:4][Xe:24].[CH3:5][O:6][P:7](=[O:8])([OH:9])[O:10][P:11](=[O:12])([OH:13])[O:14][P:15](=[O:16])([OH:17])[OH:18].[O:19]=[C:20]([OH:21])[OH:22]>>[CH2:1]([C:2](=[O:3])[S:4][Xe:24])[C:20](=[O:19])[OH:22].[CH3:5][O:6][P:7](=[O:8])([OH:9])[O:10][P:11](=[O:12])([OH:13])[OH:21].[H:23][O:14][P:15](=[O:16])([OH:17])[OH:18]","ATP + acetyl-CoA + HCO3- = ADP + malonyl-CoA + phosphate ","6.4.1.2"]
      ]

# RULES =[["[CH3:1][C:2](=[O:3])[S:4][Xe:1000].[NH2:5][c:6]1[cH:7][cH:8][c:9]([C:10](=[O:11])[OH:12])[cH:13][cH:14]1>>[S:4]([H:1001])[Xe:1000].[CH3:1][C:2](=[O:3])[NH:5][c:6]1[cH:7][cH:8][c:9]([C:10](=[O:11])[OH:12])[cH:13][cH:14]1","acetyl-CoA + 4-aminobenzoic acid = CoA + N-acetyl-4-aminobenzoate","2.3.1.50"],
#       ["[CH3:1][C:2](=[O:3])[S:4][Xe:1000].[NH2:5][c:6]1[cH:7][cH:8][c:9]([OH:10])[cH:11][cH:12]1>>[S:4]([H:1001])[Xe:1000].[CH3:1][C:2](=[O:3])[NH2:5][c:6]1[cH:7][cH:8][c:9]([OH:10])[cH:11][cH:12]1","acetyl-CoA + 4-amimophenol = CoA + paracetamol","2.3.1.118"],
# ["[H+].[H:1000][U:1001].[NH2:1][c:2]1[cH:3][cH:4][c:5]([C:6](=[O:7])[OH:8])[cH:9][cH:10]1.[O:11]=[O:12]>>[C:6](=[O:7])=[O:8].[U+:1001].[OH2:12].[NH2:1][c:2]1[cH:3][cH:4][c:5]([OH:11])[cH:9][cH:10]1","4-aminobenzoate + NAD(P)H + H+ + O2 = 4-aminophenol + NAD(P)+ + H2O + CO2","1.14.13.27"],]



RULES_ARR = np.array(RULES)
rules_set = RULES_ARR[:,:2]
RULES_DF = pd.DataFrame(RULES,columns=["mapped","orig_rxn_text","EC"])
RULES_DF 

,mapped,orig_rxn_text,EC
0,[CH3:1][C:2]([H:8])([O:3][H:7])[C:4](=[O:5])[O...,(S)-lactate + NAD+ = pyruvate + NADH + H+,1.1.1.27
1,[CH3:1][C:2]([H:6])=[O:3].[*+:5].[S:4]([H:8])[...,acetaldehyde + CoA + NAD+ = acetyl-CoA + NADH ...,1.2.1.10
2,[H:20][N:1]([H:21])[C:2]([CH2:3][C:4](=[O:5])[...,L-Aspartate + 2-Oxoglutarate <=> Oxaloacetate ...,2.6.1.1
3,[CH3:1](=[O:2])[S:3][*:5].[H:7][O:4][H:6]>>[CH...,Acyl-CoA + H2O <=> CoA + Carboxylate,3.1.2.20
4,[CH3:1][S+:2]([CH2:3][CH2:4][CH:5]([H:11])[NH2...,S-adenosyl-L-methionine = (5-deoxy-5-adenosyl)...,4.1.1.50
5,[O:1]=[C:2]([OH:3])[CH:4]([H:11])[C@H:5]([O:6]...,fumarate + H2O = L-malate,4.2.1.2
6,[H:8][CH2:1][CH:2]([O:3][H:9])[C:4](=[O:5])[S:...,Lactoyl-CoA <=> Propenoyl-CoA + H2O,4.2.1.54
7,[H:13][NH2:1][C@@H:2]([CH:3]([H:10])[C:4](=[O:...,L-Aspartate <=> Fumarate + Ammonia,4.3.1.1
8,[NH2:1][CH2:2][CH2:3][CH2:4][CH:5]([H:11])[C@H...,"L-lysine = (3S)-3,6-diaminohexanoate",5.4.3.2
9,[CH3:1][C:2](=[O:3])[OH:4].[Xe:18][S:5][H:19]....,ATP + Acetate + CoA <=> AMP + Diphosphate + Ac...,6.2.1.1


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import GENConv, GINEConv
from torch_geometric.data import Batch
from torch_scatter import scatter_add
from torch_geometric.utils import to_dense_batch

class GraphGPSLayer(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.dim_h = cfg.dim_h
        self.num_heads = cfg.num_heads
        self.attn_dropout = cfg.attn_dropout
        self.layer_norm = cfg.layer_norm
        self.batch_norm = cfg.batch_norm

        # Local message-passing model.
        if cfg.local_gnn_type == 'None':
            self.local_model = None
        elif cfg.local_gnn_type == 'GENConv':
            self.local_model = GENConv(cfg.dim_h, cfg.dim_h)
        elif cfg.local_gnn_type == 'GINE':
            self.local_model = GINEConv(nn.Sequential(nn.Linear(cfg.dim_h, cfg.dim_h), nn.GELU()), edge_dim=cfg.dim_h)

        # Global attention transformer-style model.
        if cfg.global_model_type == 'None':
            self.self_attn = None
        elif cfg.global_model_type == 'Transformer':
            self.self_attn = torch.nn.MultiheadAttention(cfg.dim_h, cfg.num_heads, dropout=self.attn_dropout, batch_first=True)

        if self.layer_norm and self.batch_norm:
            raise ValueError("Cannot apply two types of normalization together")

        # Normalization for MPNN and Self-Attention representations.
        if self.layer_norm:
            self.norm1_local = nn.LayerNorm(cfg.dim_h)
            self.norm1_attn = nn.LayerNorm(cfg.dim_h)
        if self.batch_norm:
            self.norm1_local = nn.BatchNorm1d(cfg.dim_h)
            self.norm1_attn = nn.BatchNorm1d(cfg.dim_h)

        self.dropout_local = nn.Dropout(cfg.dropout)
        self.dropout_attn = nn.Dropout(cfg.dropout)

        # Feed Forward block.
        self.activation = F.gelu
        self.ff_linear1 = nn.Linear(cfg.dim_h, cfg.dim_h * 2)
        self.ff_linear2 = nn.Linear(cfg.dim_h * 2, cfg.dim_h)
        if self.layer_norm:
            self.norm2 = nn.LayerNorm(cfg.dim_h)
        if self.batch_norm:
            self.norm2 = nn.BatchNorm1d(cfg.dim_h)
        self.ff_dropout1 = nn.Dropout(cfg.dropout)
        self.ff_dropout2 = nn.Dropout(cfg.dropout)

    def forward(self, batch):
        h = batch.x

        h_in1 = h  # for first residual connection
        h_out_list = []

        # Local MPNN with edge attributes.
        if self.local_model is not None:
            h_local = self.local_model(h, batch.edge_index, edge_attr=batch.edge_attr.float())
            h_local = self.dropout_local(h_local)
            h_local = h_in1 + h_local  # Residual connection.
            if self.layer_norm:
                h_local = self.norm1_local(h_local)
            if self.batch_norm:
                h_local = self.norm1_local(h_local)
            h_out_list.append(h_local)

        # Multi-head attention or compression.
        if self.self_attn is not None:
            h_dense, mask = to_dense_batch(h, batch.batch)
            h_attn = self._sa_block(h_dense, mask)
            h_attn = self.dropout_attn(h_attn)
            h_attn = h_in1 + h_attn  # Residual connection.
            if self.layer_norm:
                h_attn = self.norm1_attn(h_attn)
            if self.batch_norm:
                h_attn = self.norm1_attn(h_attn)
            h_out_list.append(h_attn)

        # Combine local and global outputs.
        h = sum(h_out_list)

        # Feed Forward block.
        h = h + self._ff_block(h)
        if self.layer_norm:
            h = self.norm2(h)
        if self.batch_norm:
            h = self.norm2(h)

        batch.x = h
        return batch

    def _sa_block(self, x, mask):
        """Self-attention block."""
        x = self.self_attn(x, x, x, attn_mask=mask, need_weights=False)[0]
        return x

    def _ff_block(self, x):
        """Feed Forward block."""
        x = self.ff_dropout1(self.activation(self.ff_linear1(x)))
        return self.ff_dropout2(self.ff_linear2(x))

class GPSModel(torch.nn.Module):
    """General-Powerful-Scalable graph transformer.
    https://arxiv.org/abs/2205.12454
    Rampasek, L., Galkin, M., Dwivedi, V. P., Luu, A. T., Wolf, G., & Beaini, D.
    Recipe for a general, powerful, scalable graph transformer. (NeurIPS 2022)
    """

    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.encoder = nn.Linear_pyg(dim_in, dim_out)
        dim_in = self.encoder.dim_in

        # if cfg.gnn.layers_pre_mp > 0:
        #     self.pre_mp = GNNPreMP(
        #         dim_in, cfg.gnn.dim_inner, cfg.gnn.layers_pre_mp)
        #     dim_in = cfg.gnn.dim_inner

        # if not cfg.gt.dim_hidden == cfg.gnn.dim_inner == dim_in:
        #     raise ValueError(
        #         f"The inner and hidden dims must match: "
        #         f"embed_dim={cfg.gt.dim_hidden} dim_inner={cfg.gnn.dim_inner} "
        #         f"dim_in={dim_in}"
        #     )

        try:
            local_gnn_type, global_model_type = cfg.gt.layer_type.split('+')
        except:
            raise ValueError(f"Unexpected layer type: {cfg.gt.layer_type}")
        layers = []
        for _ in range(cfg.gt.layers):
            layers.append(GPSLayer(
                dim_h=cfg.gt.dim_hidden,
                local_gnn_type=local_gnn_type,
                global_model_type=global_model_type,
                num_heads=cfg.gt.n_heads,
                act=cfg.gnn.act,
                pna_degrees=cfg.gt.pna_degrees,
                equivstable_pe=cfg.posenc_EquivStableLapPE.enable,
                dropout=cfg.gt.dropout,
                attn_dropout=cfg.gt.attn_dropout,
                layer_norm=cfg.gt.layer_norm,
                batch_norm=cfg.gt.batch_norm,
                bigbird_cfg=cfg.gt.bigbird,
                log_attn_weights=cfg.train.mode == 'log-attn-weights',
            ))
        self.layers = torch.nn.Sequential(*layers)

        GNNHead = register.head_dict[cfg.gnn.head]
        self.post_mp = GNNHead(dim_in=cfg.gnn.dim_inner, dim_out=dim_out)

    def forward(self, batch):
        for module in self.children():
            batch = module(batch)
        return batch

# Create an instance of MyModel
model = MyModel()

# Accessing child modules using self.children()
for child_module in model.children():
    print(child_module)


Linear(in_features=10, out_features=20, bias=True)
Linear(in_features=20, out_features=1, bias=True)


In [3]:

reaction = AllChem.ReactionFromSmarts(rules_set[0][0])
rxn_img = Draw.ReactionToImage(reaction, subImgSize=(1200, 1200))
rxn_img.show()
# rl = smarts_to_gml([rxn,"(S)-lactate + NAD+ = pyruvate + NADH + H+"])
# print(rl)

In [17]:
mol = Chem.MolFromSmiles("[*+:1][O:2]")
for atom in mol.GetAtoms():
    print(atom.GetSymbol(),atom.GetIdx(),atom.GetFormalCharge())

    print(atom.GetAtomicNum(),atom.GetChiralTag(),atom.GetHybridization())
    print(atom.GetIsAromatic(),atom.IsInRing(),atom.GetAtomMapNum())

* 0 1
0 CHI_UNSPECIFIED UNSPECIFIED
False False 1
O 1 0
8 CHI_UNSPECIFIED SP3
False False 2


In [8]:
rules = []

for r in rules_set:
    rule = smarts_to_gml(r)
    rules.append(rule)
    print(rule)

for i, r in enumerate(rules):

    with open(f"/home/mescalin/yitao/Documents/Code/rules_{i}.gml","w") as f:

        f.write(r)

rule[
 ruleID "acetyl-CoA + 4-aminobenzoic acid = CoA + N-acetyl-4-aminobenzoate" 
 left [
   edge [ source 2 target 4 label "-"]
 ] 
 context [
   node [ id 1 label "C"]
   node [ id 2 label "C"]
   node [ id 3 label "O"]
   node [ id 4 label "S"]
   node [ id 5 label "N"]
   node [ id 6 label "C"]
   node [ id 7 label "C"]
   node [ id 8 label "C"]
   node [ id 9 label "C"]
   node [ id 10 label "C"]
   node [ id 11 label "O"]
   node [ id 12 label "O"]
   node [ id 13 label "C"]
   node [ id 14 label "C"]
   node [ id 1000 label "Xe"]
   edge [ source 1 target 2 label "-"]
   edge [ source 2 target 3 label "="]
   edge [ source 4 target 1000 label "-"]
   edge [ source 5 target 6 label "-"]
   edge [ source 6 target 7 label ":"]
   edge [ source 6 target 14 label ":"]
   edge [ source 7 target 8 label ":"]
   edge [ source 8 target 9 label ":"]
   edge [ source 9 target 10 label "-"]
   edge [ source 9 target 13 label ":"]
   edge [ source 10 target 11 label "="]
   edge [ source 10

In [6]:
rgml = []

for r in rules:
    rgml.append(ruleGMLString(r))


rgml

['(S)-lactate + NAD+ = pyruvate + NADH + H+'(0),
 'acetaldehyde + CoA + NAD+ = acetyl-CoA + NADH + H+'(1),
 'L-Aspartate + 2-Oxoglutarate <=> Oxaloacetate + L-Glutamate'(2),
 'Acyl-CoA + H2O <=> CoA + Carboxylate'(3),
 'S-adenosyl-L-methionine = (5-deoxy-5-adenosyl)(3-aminopropyl)methylsulfonium salt + CO2'(4),
 'fumarate + H2O = L-malate'(5),
 'Lactoyl-CoA <=> Propenoyl-CoA + H2O'(6),
 'L-Aspartate <=> Fumarate + Ammonia'(7),
 'L-lysine = (3S)-3,6-diaminohexanoate'(8),
 'ATP + Acetate + CoA <=> AMP + Diphosphate + Acetyl-CoA'(9),
 'ATP + acetyl-CoA + HCO3- = ADP + malonyl-CoA + phosphate '(10)]

In [7]:
pyruvate = smiles("CC(=O)C(=O)[O-]", "PYRUVATE")
lactate = smiles("CC(C(=O)[O-])O", "LACTATE")
# lactoyl_coa = smiles("CC(C(=O)SCCNC(=O)CCNC(=O)[C@@H](C(C)(C)COP(=O)(O)OP(=O)(O)OC[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)(O)O)O)O", "LACTOYL-COA")
# aspartate = smiles("C([C@@H](C(=O)O)N)C(=O)O", "ASPARTATE")
nadh = graphDFS("[U][H]")
nadplus = graphDFS("[U+]")
oxygen = smiles("O=O", "O2")
water = smiles("O", "H2O")
hplus = smiles("[H+]", "hplus")

In [8]:
knownMols = [pyruvate, lactate,nadh,nadplus,oxygen,water,hplus]
# lactoyl_coa, /
# aspartate
knownMols

['PYRUVATE'(0),
 'LACTATE'(1),
 'g_{2}'(2),
 'g_{3}'(3),
 'O2'(4),
 'H2O'(5),
 'hplus'(6)]

In [ ]:
# flush summary file handle
post.flushCommands()
# generate summary/summery.pdf
subprocess.run(["/home/talax/xtof/local/Mod/bin/mod_post"])

In [ ]:

eductMols = [pyruvate, lactate, nadh,nadplus, water]

strat = (addSubset(eductMols)>> repeat[5](inputRules))
ls = LabelSettings(LabelType.Term, LabelRelation.Unification)
dg = DG(graphDatabase=knownMols, labelSettings=ls)
dg

In [51]:
dg.build().execute(strat)
dg


Repeat, limit = 5
  Round 1:
  Round 1: Result subset has 0 graphs.


DG(4)

In [54]:
dg.print()


LogicError: Can not open file 'out/002_dg_4_11100.dot'.

In [4]:
import torch
from torch_geometric.nn import HeteroLinear

# Sample input data
features = torch.tensor([[1.0, 2.0],  # Person node
                         [3.0, 4.0],  # Company node
                         [5.0, 6.0],  # Person node 
                         [7.0, 8.0]]) # Location node
type_vec = torch.tensor([0, 1, 0, 2])  # 0 for Person, 1 for Company, 2 for Location

# Create a HeteroLinear layer
hetero_linear = HeteroLinear(in_channels=2, out_channels=3, num_types=3)

# Apply the transformation
output = hetero_linear(features, type_vec)
print(output)


/scr/xindus/yitao/miniforge3/envs/aibio/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[ 0.7389,  2.0832, -0.4622],
        [ 3.3329, -0.7149, -3.9373],
        [ 1.5288,  6.1248, -3.6399],
        [ 2.5763, -6.6367,  6.8477]], grad_fn=<IndexPutBackward0>)
